**Extraction data from zip files**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1 : Extraction

we will extaract the data and use only 2000 of the total data to make it faster and we will try to reach high accuracy 

In [ ]:
import zipfile
original_bath ='/kaggle/input/dogs-vs-cats/train.zip'
directory_to_extract_to= "/kaggle/working/data"
with zipfile.ZipFile(original_bath, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
import zipfile
original_bath ='/kaggle/input/dogs-vs-cats/test1.zip'
directory_to_extract_to= "/kaggle/working/data"
with zipfile.ZipFile(original_bath, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
import os , shutil


original_bath = "/kaggle/working/data"
base_dir = '/kaggle/working//cats_and_dogs_small'
train_dir = os.path.join(base_dir , 'train')
validation_dir = os.path.join(base_dir , 'validation')
test_dir = os.path.join(base_dir , 'test')

os.mkdir(base_dir)

os.mkdir(train_dir)
os.mkdir(validation_dir)
os.mkdir(test_dir)



In [ ]:
train_cats_dir = os.path.join(train_dir , 'cats')
train_dogs_dir = os.path.join(train_dir , 'dogs')
os.mkdir(train_cats_dir)
os.mkdir(train_dogs_dir)


validation_cats_dir = os.path.join(validation_dir , 'cats')
validation_dogs_dir = os.path.join(validation_dir , 'dogs')
os.mkdir(validation_cats_dir)
os.mkdir(validation_dogs_dir)


test_cats_dir = os.path.join(test_dir , 'cats')
test_dogs_dir = os.path.join(test_dir , 'dogs')
os.mkdir(test_cats_dir)
os.mkdir(test_dogs_dir)


In [ ]:
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames :
    src  = os.path.join(original_bath+'/train' , fname)
    dst = os.path.join(train_cats_dir , fname)
    shutil.copyfile(src , dst)
fnames = ['cat.{}.jpg'.format(i) for i in range(1000 , 1500)]
for fname in fnames :
    src  = os.path.join(original_bath+'/train' , fname)
    dst = os.path.join(validation_cats_dir , fname)
    shutil.copyfile(src , dst)
fnames = ['cat.{}.jpg'.format(i) for i in range(1500 , 2000)]
for fname in fnames :
    src  = os.path.join(original_bath+'/train' , fname)
    dst = os.path.join(test_cats_dir , fname)
    shutil.copyfile(src , dst)


In [ ]:
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames :
    src  = os.path.join(original_bath+'/train' , fname)
    dst = os.path.join(train_dogs_dir , fname)
    shutil.copyfile(src , dst)
fnames = ['dog.{}.jpg'.format(i) for i in range(1000 , 1500)]
for fname in fnames :
    src  = os.path.join(original_bath+'/train' , fname)
    dst = os.path.join(validation_dogs_dir , fname)
    shutil.copyfile(src , dst)
fnames = ['dog.{}.jpg'.format(i) for i in range(1500  , 2000)]
for fname in fnames :
    src  = os.path.join(original_bath+'/train' , fname)
    dst = os.path.join(test_dogs_dir , fname)
    shutil.copyfile(src , dst)


In [ ]:
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dogs images:', len(os.listdir(train_dogs_dir)))

print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dogs images:', len(os.listdir(validation_dogs_dir)))

print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dogs images:', len(os.listdir(test_dogs_dir)))


In [ ]:
from keras import layers
from keras import models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())

model.add(layers.Dropout(.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()


In [ ]:
from keras import optimizers

model.compile(loss = "binary_crossentropy" , optimizer=optimizers.RMSprop(learning_rate=1e-4) ,metrics=['acc'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator



train_datagen = ImageDataGenerator(rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
                                   )
test_datagen = ImageDataGenerator(rescale=1./255)




train_generator = train_datagen.flow_from_directory(
    train_dir , 
    batch_size=32 , 
    class_mode='binary',
    target_size=(150 , 150)
)
vlidation_generator = test_datagen.flow_from_directory(
    validation_dir , 
    batch_size=32 , 
    target_size=(150 , 150),

    class_mode='binary'
)

In [ ]:
for data_batch , labels_batch in train_generator:
    print('data batch shape' , data_batch.shape)
    print('labels batch shape' , labels_batch.shape)
    break

In [ ]:
history = model.fit(
    train_generator , 
    steps_per_epoch=20 , 
    epochs=100 , 
    validation_data=vlidation_generator , 
    validation_steps=10
)


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc= history.history['val_acc']
loss = history.history['loss']
val_loss= history.history['val_loss']

epochs = range(1 , len(acc)+1)
plt.plot(epochs , acc , 'bo' , label = "Training acc")
plt.plot(epochs , val_acc , 'b' , label ="Validation acc")
plt.title("Training and Validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs , loss , 'bo' , label = "Training loss")
plt.plot(epochs , val_loss , 'b' , label ="Validation loss")
plt.title("Training and Validation loss")
plt.legend()

In [ ]:
##using VGG 16 

from keras.applications import VGG16
conv_base = VGG16(weights = 'imagenet' , 
                  include_top =False , 
                  input_shape=(150 , 150 , 3))

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20
def extract_features (direcrory , sample_count):
    ##(4,4,512) because the last shape in vgg is (4,4,512)
    features =np.zeros(shape =(sample_count , 4,4,512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        direcrory, 
        target_size=(150 , 150),
        batch_size=batch_size,
        class_mode='binary'

    )
    i=0
    for input_batch , labels_batch in generator:
        feature_batch = conv_base.predict(input_batch)
        features[i*batch_size :(i+1) * batch_size] =feature_batch
        labels[i*batch_size : (i+1)*batch_size]= labels_batch
        i+=1
        if i*batch_size >= sample_count:
            break
    return features , labels

In [ ]:
train_features , train_labels = extract_features(train_dir , 2000)
validation_features , validation_labels = extract_features(validation_dir , 1000)
test_features , test_labels = extract_features(test_dir , 1000)



In [ ]:
from keras import layers , models

train_features = np.reshape(train_features , (2000 , 4*4*512))
validation_features = np.reshape(validation_features , (1000 , 4*4*512))
test_features = np.reshape(test_features , (1000 , 4*4*512))


In [ ]:
model = models.Sequential()
model.add(layers.Dense(256 , activation="relu" , input_dim = 512*4*4))
model.add(layers.Dropout(.5))
model.add(layers.Dense(1 ,activation="sigmoid"))


In [ ]:
from keras import optimizers
model.compile(loss='binary_crossentropy' , optimizer=optimizers.RMSprop(lr=1e-4) ,metrics=['acc'])

In [ ]:
history = model.fit(train_features, train_labels , 
                    epochs=30 , 
                    validation_data=(validation_features , validation_labels))

In [ ]:
import matplotlib.pyplot as plt
acc= history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1 , len(acc)+1)
plt.plot(epochs , acc ,'bo' , label = "Training acc")
plt.plot(epochs , val_acc ,'b' , label = "Validation acc")
plt.legend()
plt.figure()

plt.plot(epochs , loss ,'bo' , label = "Training loss")
plt.plot(epochs , val_loss ,'b' , label = "Validation loss")
plt.legend()
plt.show()


In [ ]:
from keras import layers , models
model  = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256 , activation="relu"))
model.add(layers.Dense(1 , activation="sigmoid"))

In [ ]:
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
train_dir,
target_size=(150, 150),
batch_size=20,
class_mode='binary')
validation_generator = test_datagen.flow_from_directory(
validation_dir,
target_size=(150, 150),
batch_size=20,
class_mode='binary')
model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(lr=2e-5),
metrics=['acc'])
history = model.fit_generator(
train_generator,
steps_per_epoch=100,
epochs=30,
validation_data=validation_generator,
validation_steps=50)

In [ ]:
import matplotlib.pyplot as plt
acc= history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1 , len(acc)+1)
plt.plot(epochs , acc ,'bo' , label = "Training acc")
plt.plot(epochs , val_acc ,'b' , label = "Validation acc")
plt.legend()
plt.figure()

plt.plot(epochs , loss ,'bo' , label = "Training loss")
plt.plot(epochs , val_loss ,'b' , label = "Validation loss")
plt.legend()
plt.show()


In [ ]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
model.compile(loss ='binary_crossentropy' , 
              optimizer=optimizers.RMSprop(lr=1e-4) , 
            metrics=['acc'])





In [ ]:
history = model.fit(train_generator , 
                    epochs=100 , 
                    steps_per_epoch=100 ,
                    validation_data=validation_generator , 
                    validation_steps=50)


In [ ]:
import matplotlib.pyplot as plt
acc= history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1 , len(acc)+1)
plt.plot(epochs , acc ,'bo' , label = "Training acc")
plt.plot(epochs , val_acc ,'b' , label = "Validation acc")
plt.legend()
plt.figure()

plt.plot(epochs , loss ,'bo' , label = "Training loss")
plt.plot(epochs , val_loss ,'b' , label = "Validation loss")
plt.legend()
plt.show()


In [ ]:
def smooth_curve(points , factor =.8):
    somoothed_points = []
    for point in points:
        if somoothed_points:
            previous = somoothed_points[-1]
            somoothed_points.append(previous *factor + point *(1-factor))
        else:
            somoothed_points.append(point)
    return somoothed_points


acc= history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(epochs,
smooth_curve(acc), 'bo', label='Smoothed training acc')
plt.plot(epochs,
smooth_curve(val_acc), 'b', label='Smoothed validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs,
smooth_curve(loss), 'bo', label='Smoothed training loss')
plt.plot(epochs,
smooth_curve(val_loss), 'b', label='Smoothed validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
test_generator = test_datagen.flow_from_directory(
test_dir,
target_size=(150, 150),
batch_size=20,
class_mode='binary')
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)